Import libraries

In [11]:
import os
import re
import json
import sqlite3
import chromadb
from chromadb.utils import embedding_functions
from dotenv import load_dotenv
import google.generativeai as genai


Load API key from .env


In [2]:
load_dotenv()
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

Defining Paths

In [3]:
EXAMPLES_PATH = "data/examples.json"
DB_PATH = "data/SuperStoreOrders.db" 

Load examples.json file

In [4]:
with open(EXAMPLES_PATH, "r") as f:
    examples = json.load(f)

Chroma DB for RAG

In [5]:
embedding_fn = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")
client = chromadb.Client()
collection = client.create_collection(name="patterns", embedding_function=embedding_fn)

# Add examples to vector DB (store checklist as JSON string)

for idx, ex in enumerate(examples):
    collection.add(
        ids=[str(idx)],
        documents=[ex["pattern"]],
        metadatas=[{"checklist": json.dumps(ex["checklist"])}]  # store as string
    )


/Users/gokul/miniconda3/envs/rca/lib/python3.13/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Function to retrieve similar patterns

In [6]:
def retrieve_similar_patterns(user_pattern, top_k=2):
    results = collection.query(query_texts=[user_pattern], n_results=top_k)
    # Parse checklist string back to list
    parsed_results = []
    for meta in results["metadatas"]:
        checklist = json.loads(meta[0]["checklist"]) if meta[0].get("checklist") else []
        parsed_results.append({"checklist": checklist})
    return parsed_results

Function to generate checklist, given new pattern

In [17]:
def generate_checklist(user_pattern, retrieved_patterns):
    prompt = f"""
You are a root cause analysis assistant.
Given the following historical patterns and their checklists:
{retrieved_patterns}

Now, create a final checklist (only 5 points in checklist) for this new pattern:
'{user_pattern}'
Checklist should be step-by-step, concise, and relevant to database columns:
['order_id', 'order_date', 'ship_date', 'ship_mode', 'customer_name', 'segment', 'state', 'country', 'market', 'region', 'product_id', 'category', 'sub_category', 'product_name', 'sales', 'quantity', 'discount', 'profit', 'shipping_cost', 'order_priority', 'year']
    """
    resp = genai.GenerativeModel("gemini-1.5-flash").generate_content(prompt)
    return [item.strip("- ").strip() for item in resp.text.split("\n") if item.strip()]


Function to generate SQL query

In [15]:
def generate_sql(checklist_item):
    prompt = f"""
You are an expert SQL generator.
Write ONE valid SQLite query (no multiple statements, no comments, no placeholders).
Use the table 'orders' with columns:
['order_id', 'order_date', 'ship_date', 'ship_mode', 'customer_name', 'segment', 'state', 'country', 'market', 'region', 'product_id', 'category', 'sub_category', 'product_name', 'sales', 'quantity', 'discount', 'profit', 'shipping_cost', 'order_priority', 'year']

The request is:
"{checklist_item}"

Return ONLY the SQL statement, no explanation, no comments, no markdown.
"""
    resp = genai.GenerativeModel("gemini-1.5-flash").generate_content(prompt)
    return resp.text.strip()


Function to run SQL query

In [14]:

def clean_sql(sql):
    # Remove SQL comments
    sql = re.sub(r"--.*", "", sql)
    # Remove extra spaces
    sql = sql.strip()
    # Remove code block markers from Gemini output
    sql = sql.replace("```sql", "").replace("```", "").strip()
    # Ensure only one statement by splitting on semicolon
    if ";" in sql:
        sql = sql.split(";")[0]
    return sql

def run_sql(query):
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    try:
        cleaned_query = clean_sql(query)
        cursor.execute(cleaned_query)
        results = cursor.fetchall()
        col_names = [desc[0] for desc in cursor.description]
        conn.close()
        return col_names, results
    except Exception as e:
        conn.close()
        return None, str(e)


Testing with new real life pattern

In [18]:
user_input_pattern = "Sales drop in electronics category in the east region"

# Step 1: Retrieve similar patterns
similar = retrieve_similar_patterns(user_input_pattern)

# Step 2: Generate checklist
checklist = generate_checklist(user_input_pattern, similar)
print("\nGenerated Checklist:")
for i, step in enumerate(checklist, 1):
    print(f"{i}. {step}")

# Step 3: Generate and run SQL for each checklist item
for step in checklist:
    sql = generate_sql(step)
    print(f"\nChecklist Item: {step}")
    print(f"SQL:\n{sql}")
    cols, res = run_sql(sql)
    if cols:
        print("Results:", res[:5])  # Show first 5 rows
    else:
        print("Error:", res)


Generated Checklist:
1. Final Checklist: Sales Drop in Electronics Category - East Region
2. 1. **Quantify the drop:**  Calculate the MoM percentage change in `sales` for the 'Electronics' `category` in the 'East' `region` using `order_date` and `sales` columns.
3. 2. **Analyze regional sales trends:** Compare the MoM percentage change in `sales` for 'Electronics' in the 'East' `region` against other `region`s to determine if it's an isolated incident.
4. 3. **Investigate promotional activity:** Analyze `discount` levels and frequency for 'Electronics' products in the 'East' `region` using `order_date` and `discount` columns. Compare to previous periods.
5. 4. **Examine product performance:** Identify specific `product_name`s (within 'Electronics' category) with the largest sales declines in the 'East' `region` using `product_name`, `sales`, and `region` columns.
6. 5. **Assess competitor impact (if data available):** If external data is available, compare the sales trend of 'Electron